In [68]:
%run conservation_equations.ipynb
import numpy as np
import math
import matplotlib.pyplot as plt
from iapws import IAPWS97

0.0019392683356514282


In [69]:
"""Define Inlet Parameters"""
diameter = 0.7366 #29 inch ID
mass_flow = 51
pressure = 0.8862
length = 80
roughness = 0.000046 #m

In [70]:
'''Define Outlet Parameters'''
diameter2 = diameter
mass_flow2 = mass_flow

In [71]:
"""Define Fittings"""
#Represent in a list of 8
fittings_list = [0, 2, 0, 0, 0, 0, 0, 0]
#fittings [0] = elbow_45  #fittings [1] = elbow_90
#fittings [2] = tee       #fittings [3] = return_bend
#fittings [4] = coupling  #fittings [5] = union
#fittings [6] = gate_valve_full  #fittings [7] = gate_valve_half

In [72]:
'''Calculate parameters from steam table'''
sat_steam = IAPWS97(P = pressure, x=1)
temperature = sat_steam.T
density = sat_steam.rho
enthalpy =  sat_steam.h
dynamic_viscosity = sat_steam.mu
print(temperature, density, dynamic_viscosity, enthalpy)

447.85219616564257 4.585969505476386 1.4804923935624342e-05 2772.4261727237977


In [73]:
'''Calculate Derived Parameters'''
#Relative roughness
relative_roughness = roughness / diameter

#cross sectional area
area = math.pi * diameter**2 / 4
#for straight pipe
area2 = area

#velocity
velocity = mass_flow / (density * area)
print(velocity, area)

26.09670661426525 0.42614098992099564


In [74]:
'''Skin Friction Calculation'''
nRE = reynolds_number(diameter, velocity, density, dynamic_viscosity)
friction_factor = ff_colebrook(nRE, relative_roughness)
friction_skin = f_skin(temperature, diameter, velocity, relative_roughness, length)
#print('nRE',nRE)
#print(friction_skin, friction_factor)
#print(temperature, diameter, velocity, relative_roughness, length)

0.0030478696133668967
nRE 5954460.241631568
450.8748995236644 0.002839999999998332
447.85219616564257 0.7366 26.09670661426525 6.24490904154222e-05 80


In [75]:
'''Pipe Enlargement and contraction '''
friction_expansion = f_expansion(area, area2, velocity)
friction_contraction = f_contraction(area, area2, velocity)

In [76]:
'''Losses in fittings and valves'''
friction_fittings = f_fittings(velocity, fittings_list)
print (velocity, friction_fittings)

26.09670661426525 510.7785720832768


In [77]:
'''Calulate total friction losses'''
friction_total = friction_skin + friction_expansion + friction_contraction + friction_fittings
print (friction_total)

961.6534716069411


In [78]:
'''Define Parameters for heat loss'''
#Convective heat transfer coefficient (h; W/m**K)
h_steam_pipe = 36
h_insulation_air = 7
#Insulation
t_insulation = 2 * 0.0254    #insulation thickness (m)
k_insulation = 0.05          #insulation's thermal conductivity (W/m** K)
#Pipe
t_pipe = 0.5 * 0.0254         #pipeline thickness (m)
k_pipe = 56           #pipe's thermal conductivity (W/m** K)
#id_pipe = 0.25        #pipe's inner diameter (m) given at diameter
#l_pipe = 6000         #length of pipeline (m) given at length
#Temp
t1 = temperature - 273.15           #temp inside the pipeline
t2 = 25                             #temp; atmospheric

r1 = diameter
r2 = diameter + t_pipe
r3 = diameter + t_pipe + t_insulation

h_t = heat_transfer(length, t1, t2, h_steam_pipe, h_insulation_air, k_pipe, k_insulation, r1, r2, r3)
Q = h_t / mass_flow
print(Q, h_t)

-965.2933921570551 -49229.96300000981


In [79]:
'''Calculate gdZ'''
gdz = head(655, 613)
print (gdz)

412.02000000000004


In [80]:
'''Calculate outlet pressure'''
udu = 0
p_out(pressure, friction_total, gdz, udu, density, Q )

0.8792525915997513